2D Omnipose segmentation
============================

Before running this notebook, install the latest version of Omnipose from GitHub. This automatically installs our Cellpose fork. 

In [1]:
# make local editable packages automatically reload
%load_ext autoreload
%autoreload 2

In [2]:
# Import dependencies
import numpy as np
from cellpose_omni import models, core

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? {}'.format(use_GPU))

# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plt.style.use('dark_background')
%matplotlib inline

2023-12-04 22:15:27,054 [INFO] TORCH GPU version not installed/working.
>>> GPU activated? False


C:\Users\Giona\anaconda3\envs\omnipose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pathlib import Path
import os
from cellpose_omni import io
import omnipose
import imageio

# Set the path to the desired folder
#basedir = 'CellA_ShME480'

# Get the list of image files from the folder
#files = io.get_image_files(basedir)

# Path to the video file
video_path = 'CellA_ShME480.avi'

# Create a reader to read the video file
reader = imageio.get_reader(video_path, 'ffmpeg')

# Read all frames in a list
frames = [im for im in reader]

# Extract the video name (without file extension) to use for naming directories
video_name = Path(video_path).stem  # This will be 'CellA_ShME480'

# Current working directory
current_working_dir = Path(os.getcwd())

# Base directory for saving segmentation masks
base_mask_dir = current_working_dir / 'segmentation_outputs' / video_name

# Create the base directory for masks, if it does not exist
if not base_mask_dir.exists():
    base_mask_dir.mkdir(parents=True, exist_ok=True)

# Generate file paths for each frame to be saved in the masks directory
files = [base_mask_dir / f"frame_{i:04d}.tif" for i in range(len(frames))]

In [4]:
from cellpose_omni import io, transforms
from omnipose.utils import normalize99
import numpy as np
import cv2

# Assuming 'frames' contains the frames extracted from the video file
imgs = []
for frame in frames:
    # Convert BGR to grayscale if your frames are in color
    # If your frames are already grayscale, you can skip this conversion
    if len(frame.shape) == 3:  # Check if the frame is colored
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        frame_gray = frame  # If the frame is already in grayscale

    # Normalize and preprocess the frame
    frame_processed = normalize99(frame_gray)
    #frame_processed = np.pad(frame_processed, 10, 'constant', constant_values=0)  # Pad with zeros

    imgs.append(frame_processed)

nimg = len(imgs)
print('\nnumber of images:', nimg)


number of images: 4852


## Initialize model
Here we use one of the built-in model names. We will choose the `bact_phase_omni` model. 

In [5]:
from cellpose_omni import models
from cellpose_omni.models import MODEL_NAMES

model_name = 'cyto'
model = models.CellposeModel(gpu=use_GPU, model_type=model_name)

2023-12-04 22:15:36,407 [INFO] >>cyto<< model set to be used
2023-12-04 22:15:36,411 [INFO] >>>> using CPU


## Run segmentation 

In [6]:
import time
from tqdm import tqdm
import numpy as np
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu, threshold_local
from skimage.morphology import closing, square

chans = [0,0]  # this means segment based on first channel, no second channel

#n= list(range(5555, 7789)) CellB_GFP
#n=[1,] # make a list of integers to select which images you want to segment
#n = range(nimg) # or just segment them all 

# List of indices for images to segment
n = list(range(1,100))  # Replace with range(nimg) or specific indices as required

# Parameters for segmentation
params = {'channels':chans, # always define this with the model
          'rescale': None, #rescaling_factor, # upscale or downscale your images, None = no rescaling 
          'mask_threshold': 3.5, # erode or dilate masks with higher or lower values 
          'flow_threshold': 0.85, # default is .4, but only needed if there are spurious masks to clean up; slows down output
          'transparency': True, # transparency in flow output
          'omni': True, # we can turn off Omnipose mask reconstruction, not advised 
          'cluster': True, # use DBSCAN clustering
          'resample': True, # whether or not to run dynamics on rescaled grid or original grid 
          #'verbose': False, # turn on if you want to see more output 
          'tile': False, # average the outputs from flipped (augmented) images; slower, usually not needed 
          'niter': None, # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 
          'augment': False, # Can optionally rotate the image and average outputs, usually not needed 
          'affinity_seg': True, # new feature, stay tuned...
          'diameter': 60 # Default diameter

}



total_segmentation_time = 0

# Loop through each image, segment and save
for i in tqdm(n, 'Segmentation progress:'):
    tic = time.time()  # Start time for this iteration

    # Perform segmentation on the i-th image
    mask, flow, style = model.eval(imgs[i], **params)

    # Calculate segmentation time for this image
    seg_time = time.time() - tic
    total_segmentation_time += seg_time
    print(f'Segmentation time for image {i}: {seg_time}s')

    # Define the file path for saving the results
    file_path = files[i]

    # Save the masks for the i-th image
    io.save_masks([imgs[i]], [mask], [flow], [file_path],
                  png=True, tif=True, suffix='',
                  save_flows=False, save_outlines=True,
                  dir_above=0, in_folders=True,
                  save_txt=True, save_ncolor=False)

print(f'Total segmentation time for all images: {total_segmentation_time}s')

average_time_per_image = total_segmentation_time / len(n)
expected_total_time = average_time_per_image * len(imgs)
print(f'Expected total time for segmenting all images: {expected_total_time}s')


Segmentation progress::   0%|                                                                   | 0/99 [00:00<?, ?it/s]

Segmentation time for image 1: 2.2207658290863037s


Segmentation progress::   1%|▌                                                          | 1/99 [00:04<07:07,  4.36s/it]

Segmentation time for image 2: 1.4040849208831787s


Segmentation progress::   2%|█▏                                                         | 2/99 [00:06<05:07,  3.17s/it]

...

Segmentation time for image 98: 1.2902624607086182s


Segmentation progress::  99%|█████████████████████████████████████████████████████████▍| 98/99 [03:39<00:02,  2.15s/it]

Segmentation time for image 99: 1.2630298137664795s


Segmentation progress:: 100%|██████████████████████████████████████████████████████████| 99/99 [03:41<00:00,  2.23s/it]

Total segmentation time for all images: 134.70569372177124s
Expected total time for segmenting all images: 6601.939655939738s


In [7]:
import cv2
import os
from natsort import natsorted
from tqdm import tqdm
from pathlib import Path

def create_video_from_images(image_folder, output_video_folder, output_video_file, fps=1, extension='.png'):
    # Get all image paths with the specified extension and sort them
    image_paths = natsorted(
        [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(extension)]
    )

    # Check if we have any images to process
    if not image_paths:
        print(f"No images with extension {extension} found in {image_folder}.")
        return

    # Full path for the output video file
    full_output_path = os.path.join(output_video_folder, output_video_file)

    # Read the first image to determine the size
    frame = cv2.imread(image_paths[0])
    height, width, layers = frame.shape

    # Define the codec for AVI format
    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')

    # Create VideoWriter object
    video = cv2.VideoWriter(full_output_path, fourcc, fps, (width, height), isColor=True)

    # Iterate over each image and add it to the video
    for image_path in tqdm(image_paths, desc=f'Creating video {output_video_file}'):
        video.write(cv2.imread(image_path))

    # Release the video writer
    video.release()

# Define the directories for masks, cp_output, and outlines
masks_dir = base_mask_dir / 'masks'
cp_output_dir = base_mask_dir / 'cp_output'
outlines_dir = base_mask_dir / 'outlines'

video_output_dir = base_mask_dir.parent / video_name


# Create the output video file names using the video_name
output_video_name_png = f"{video_name}_masks_video_png.avi"
output_video_name_tif = f"{video_name}_masks_video_tif.avi"
output_cp_output_name = f"{video_name}_cp_output_video.avi"
output_outlines_name = f"{video_name}_outlines_video.avi"

# Create videos in the video_output_dir directory
create_video_from_images(masks_dir, video_output_dir, output_video_name_png, fps=1, extension='.png')
create_video_from_images(masks_dir, video_output_dir, output_video_name_tif, fps=1, extension='.tif')
create_video_from_images(cp_output_dir, video_output_dir, output_cp_output_name, fps=1, extension='.png')
create_video_from_images(outlines_dir, video_output_dir, output_outlines_name, fps=1, extension='.png')

Creating video CellA_ShME480_outlines_video.avi: 100%|█████████████████████████████████| 99/99 [00:01<00:00, 98.12it/s]
